# West Nile
### Starter Code

In [1]:
# Original code: Beating the Benchmark from West Nile Virus Prediction @ Kaggle by Abhihsek
# Modified by Brendan Lane
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing

%matplotlib inline

In [2]:
# Load dataset 
train = pd.read_csv('../assets/train.csv')
test = pd.read_csv('../assets/test.csv')
sample = pd.read_csv('../assets/sampleSubmission.csv')
weather = pd.read_csv('../assets/weather.csv')
spray = pd.read_csv('../assets/spray.csv')

In [3]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [4]:
# Output
y = train.WnvPresent.values

In [68]:
y.shape

(10506,)

In [69]:
# ! We may want to include codesum in final model
# ! Refer to pdf in 'assets' for codesum information

# Not using codesum for this benchmark
wx = weather.drop('CodeSum', axis=1)

In [70]:
wx.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,Sunset,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,1849,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,-,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,1850,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,-,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,1851,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [71]:
# Split station 1 and 2 and join horizontally
wx1 = wx[wx['Station']==1]
wx2 = wx[wx['Station']==2]
wx1 = wx1.drop('Station', axis=1)
wx1 = wx2.drop('Station', axis=1)
wx = wx1.merge(wx2, on='Date', suffixes=('_1', '_2'))

In [72]:
wx.head()

,Date,Tmax_1,Tmin_1,Tavg_1,Depart_1,DewPoint_1,WetBulb_1,Heat_1,Cool_1,Sunrise_1,...,Sunset_2,Depth_2,Water1_2,SnowFall_2,PrecipTotal_2,StnPressure_2,SeaLevel_2,ResultSpeed_2,ResultDir_2,AvgSpeed_2
0,2007-05-01,84,52,68,M,51,57,0,3,-,...,-,M,M,M,0.00,29.18,29.82,2.7,25,9.6
1,2007-05-02,60,43,52,M,42,47,13,0,-,...,-,M,M,M,0.00,29.44,30.08,13.3,2,13.4
2,2007-05-03,67,48,58,M,40,50,7,0,-,...,-,M,M,M,0.00,29.46,30.12,12.9,6,13.2
3,2007-05-04,78,51,M,M,42,50,M,M,-,...,-,M,M,M,0.00,29.36,30.04,10.1,7,10.4
4,2007-05-05,66,54,60,M,39,50,5,0,-,...,-,M,M,M,T,29.46,30.09,11.2,7,11.5


In [73]:
# replace some missing values and T with nan
wx = wx.replace('M', np.nan)
wx = wx.replace('-', np.nan)
wx = wx.replace('T', np.nan)
wx = wx.replace(' T', np.nan)
wx = wx.replace('  T', np.nan)

In [74]:
# Functions to extract month and day from dataset
# You can also use parse_dates of Pandas.
def create_year(x):
    return(x.split('-')[0])

def create_month(x):
    return(x.split('-')[1])

def create_day(x):
    return(x.split('-')[2])

train['month'] = train.Date.apply(create_month)
train['day'] = train.Date.apply(create_day)
train['year'] = train.Date.apply(create_year)

test['month'] = test.Date.apply(create_month)
test['day'] = test.Date.apply(create_day)
test['year'] = test.Date.apply(create_year)

In [75]:
# drop address columns
train = train.drop(['Address', 'AddressNumberAndStreet','WnvPresent', 'NumMosquitos'], axis = 1)
test = test.drop(['Id', 'Address', 'AddressNumberAndStreet'], axis = 1)

In [76]:
# Merge with weather data (on hold for now)
# train = train.merge(wx, on='Date')
# test = test.merge(wx, on='Date')
# train = train.drop(['Date'], axis = 1)
# test = test.drop(['Date'], axis = 1)

In [77]:
train.head()

,Date,Species,Block,Street,Trap,Latitude,Longitude,AddressAccuracy,month,day,year
0,2007-05-29,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,9,05,29,2007
1,2007-05-29,CULEX RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,9,05,29,2007
2,2007-05-29,CULEX RESTUANS,62,N MANDELL AVE,T007,41.994991,-87.769279,9,05,29,2007
3,2007-05-29,CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,8,05,29,2007
4,2007-05-29,CULEX RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,8,05,29,2007


In [78]:
len(np.unique(train.Species))

7

In [ ]:
# Convert categorical data to numbers
lbl = preprocessing.LabelEncoder()
lbl.fit(list(train['Species'].values) + list(test['Species'].values))
train['Species'] = lbl.transform(train['Species'].values)
test['Species'] = lbl.transform(test['Species'].values)

# lbl.fit(list(train['Street'].values) + list(test['Street'].values))
# train['Street'] = lbl.transform(train['Street'].values)
# test['Street'] = lbl.transform(test['Street'].values)

lbl.fit(list(train['Trap'].values) + list(test['Trap'].values))
train['Trap'] = lbl.transform(train['Trap'].values)
test['Trap'] = lbl.transform(test['Trap'].values)

In [79]:
# Remove block and street as they are unnecessary
train.drop(columns=['Block', 'Street'], inplace=True)

In [80]:
# Remove duplicate entries
train = train.drop_duplicates()

,Date,Species,Trap,Latitude,Longitude,AddressAccuracy,month,day,year
0,2007-05-29,2,1,41.954690,-87.800991,9,05,29,2007
1,2007-05-29,3,1,41.954690,-87.800991,9,05,29,2007
2,2007-05-29,3,8,41.994991,-87.769279,9,05,29,2007
3,2007-05-29,2,15,41.974089,-87.824812,8,05,29,2007
4,2007-05-29,3,15,41.974089,-87.824812,8,05,29,2007


In [81]:
train.isnull().sum()

Date               0
Species            0
Trap               0
Latitude           0
Longitude          0
AddressAccuracy    0
month              0
day                0
year               0
dtype: int64